In [1]:
import os
os.environ["SQLITE_BUSY_TIMEOUT"] = "30000"  # 30 segundos

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, minmax_scale, scale
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader

In [3]:
data_path= "/home/21953404Victor/Deep/drive-download-20231121T165626Z-001/HotelReservationsPreparedCleanAttributes.csv"
label_path = "/home/21953404Victor/Deep/drive-download-20231121T165626Z-001/HotelReservationsOutput.csv"

In [4]:
features  = pd.read_csv(data_path)
labels = pd.read_csv(label_path)

In [5]:
from sklearn.model_selection import train_test_split
# Divide el conjunto de datos
features_train, features_test, labels_train, labels_test = train_test_split(features.values, labels.values, test_size=0.1, random_state=42)
features_train, features_val, labels_train, labels_val = train_test_split(features_train, labels_train, test_size=0.1, random_state=42)

# Convierte los arrays a tensores de PyTorch
features_train = torch.tensor(features_train).float()
labels_train = torch.tensor(labels_train).float()
features_val = torch.tensor(features_val).float()
labels_val = torch.tensor(labels_val).float()
features_test = torch.tensor(features_test).float()
labels_test = torch.tensor(labels_test).float()

# Crea conjuntos de datos de PyTorch
train_dataset = TensorDataset(features_train, labels_train)
val_dataset = TensorDataset(features_val, labels_val)
test_dataset = TensorDataset(features_test, labels_test)

# Crea DataLoaders de PyTorch
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [6]:
import tensorflow as tf
import torch

# Verificar si CUDA está disponible
cuda_available = torch.cuda.is_available()
print(f"CUDA disponible: {cuda_available}")

# Enumerar dispositivos de GPU disponibles y mostrar sus nombres si CUDA está disponible
if cuda_available:
    print("Listado de dispositivos GPU disponibles:")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No se detectó una GPU. Asegúrate de que el acelerador de hardware esté configurado.")


2023-12-01 16:41:33.989546: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-01 16:41:34.116881: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 16:41:34.692035: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/hadoop/lib/native:/opt/hadoop/lib/native:
2023-12-01 16:41:34.692092: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvin

CUDA disponible: True
Listado de dispositivos GPU disponibles:
GPU 0: NVIDIA GeForce RTX 3080 Ti


In [7]:
import torch.nn as nn

class PyTorchModel(nn.Module):
    def __init__(self, trial, input_shape):
        super(PyTorchModel, self).__init__()
        self.layers = nn.ModuleList()
        self.l2_regs = []
        current_shape = input_shape

        # Definir la arquitectura de la red
        n_layers = trial.suggest_int('n_layers', 1, 50)
        for i in range(n_layers):
            num_units = trial.suggest_categorical(f'n_units_l{i}', [16, 32, 64, 128, 256, 512, 1024, 2048])
            activation = trial.suggest_categorical(f'activation_l{i}', ['ReLU', 'Tanh', 'ELU', 'SELU', 'LeakyReLU'])
            dropout_rate = trial.suggest_float(f'dropout_l{i}', 0.0, 0.5)
            use_batch_norm = trial.suggest_categorical(f'batch_norm_l{i}', [True, False])
            l2_reg = trial.suggest_float(f'l2_reg_l{i}', 1e-5, 1e-1, log=True)

            # Crear y añadir la capa lineal
            layer = nn.Linear(current_shape, num_units)
            self.layers.append(layer)
            self.l2_regs.append(l2_reg)  # Añadir la tasa de regularización L2

            current_shape = num_units

            # Añadir Batch Normalization si se indica
            if use_batch_norm:
                self.layers.append(nn.BatchNorm1d(num_units))
            
            # Añadir Dropout
            self.layers.append(nn.Dropout(dropout_rate))

            # Añadir la activación
            if activation == 'LeakyReLU':
                negative_slope = trial.suggest_float(f'leakyrelu_alpha_l{i}', 0.01, 0.3)
                self.layers.append(nn.LeakyReLU(negative_slope=negative_slope))
            else:
                self.layers.append(getattr(nn, activation)())

        # Capa de salida
        self.output = nn.Linear(current_shape, 1)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return torch.sigmoid(self.output(x))

    def get_l2_loss(self):
        l2_loss = 0.0
        for layer, l2_reg in zip(self.layers, self.l2_regs):
            if isinstance(layer, nn.Linear):
                l2_loss += l2_reg * layer.weight.norm(2)
        return l2_loss



In [8]:
from torch.optim import Adam, SGD, RMSprop
from torch.utils.data import DataLoader

def build_model(params, input_shape, is_trial=True):
    # Crear una instancia del modelo con la arquitectura definida
    model = PyTorchModel(params, input_shape)

    if is_trial:
        # Configuración durante la optimización con Optuna
        optimizer_name = params.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop'])
        lr = params.suggest_float('lr', 1e-5, 1e-1, log=True)
        if optimizer_name == 'Adam':
            beta1 = params.suggest_float('adam_beta1', 0.85, 0.99)
            beta2 = params.suggest_float('adam_beta2', 0.9, 0.999)
            optimizer = Adam(model.parameters(), lr=lr, betas=(beta1, beta2))
        elif optimizer_name == 'SGD':
            momentum = params.suggest_float('sgd_momentum', 0.1, 0.9)
            optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
        elif optimizer_name == 'RMSprop':
            alpha = params.suggest_float('rmsprop_alpha', 0.85, 0.99)
            optimizer = RMSprop(model.parameters(), lr=lr, alpha=alpha)
    else:
        # Configuración con parámetros fijos
        optimizer_name = params['optimizer']
        if optimizer_name == 'Adam':
            optimizer = Adam(model.parameters(), lr=params['lr'], betas=(params['adam_beta1'], params['adam_beta2']))
        elif optimizer_name == 'SGD':
            optimizer = SGD(model.parameters(), lr=params['lr'], momentum=params.get('sgd_momentum', 0.9))
        elif optimizer_name == 'RMSprop':
            optimizer = RMSprop(model.parameters(), lr=params['lr'], alpha=params.get('rmsprop_alpha', 0.99))

    return model, optimizer



In [9]:
from tqdm import tqdm
import torch

def calculate_accuracy(outputs, labels):
    predicted = outputs.round()  # Redondear para obtener las predicciones binarias
    correct = (predicted == labels).sum().item()
    return correct / labels.size(0)

In [10]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import gc
from torch.utils.data import DataLoader

def plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies, file_name_base):
    plt.figure(figsize=(12, 4))

    # Pérdida
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Pérdida durante el entrenamiento')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.legend()

    # Precisión
    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.title('Precisión durante el entrenamiento')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión')
    plt.legend()

    plt.savefig(f"{file_name_base}_plot.png")
    plt.close()

In [11]:
import json

best_accuracy_file = "global_best_accuracy.json"

# Intenta cargar la mejor precisión global desde un archivo
try:
    with open(best_accuracy_file, "r") as file:
        global_best_accuracy = json.load(file)
except (FileNotFoundError, json.JSONDecodeError, TypeError, ValueError):
    global_best_accuracy = 0.0
    
best_model_path = "best_model.pt"
plot_file_name_base = "best_model"

print(f"Mejor accuracy global: {global_best_accuracy}")

Mejor accuracy global: 0.8921898928024502


## Train

In [12]:
import torch
import gc
from torch.utils.data import DataLoader

def objective(trial):
    global global_best_accuracy, best_model_path, plot_file_name_base
    # Verificar y configurar el uso de la GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Usando {'GPU' if device.type == 'cuda' else 'CPU'} para el Trial {trial.number}")

    model, optimizer = build_model(trial, input_shape)  # Asegúrate de que 'input_shape' esté definido
    model.to(device)

    # Parametrización del número de épocas y tamaño del batch
    epochs = trial.suggest_int('epochs', 10, 2000)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256, 512, 1024, 2048])

    # Criterio de pérdida y DataLoader
    criterion = torch.nn.BCELoss()  # Ajustar según tu problema
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Configuración de Early Stopping y Learning Rate Scheduler
    early_stopping_patience = trial.suggest_int('early_stopping_patience', 3, 20)
    factor = trial.suggest_float("lr_reduce_factor", 0.1, 0.9)
    lr_patience = trial.suggest_int("lr_reduce_patience", 5, 50)
    min_lr = trial.suggest_float("min_lr", 1e-6, 1e-4, log=True)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=factor, patience=lr_patience, min_lr=min_lr, verbose=True)

    # Entrenamiento del modelo
    print(f"Entrenando Trial {trial.number}...")
    best_accuracy = 0.0  # Inicializacion de la mejor precisión
    epochs_no_improve = 0
    # Inicializar listas para almacenar métricas
    train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            train_correct += calculate_accuracy(outputs, labels) * inputs.size(0)
            train_total += labels.size(0)

        train_loss /= train_total
        train_accuracy = train_correct / train_total

        # Almacenar las métricas de entrenamiento
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        # Evaluación y Early Stopping
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item() * inputs.size(0)
                val_correct += calculate_accuracy(outputs, labels) * inputs.size(0)
                val_total += labels.size(0)

        val_loss /= val_total
        val_accuracy = val_correct / val_total

        # Almacenar las métricas de validación
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        lr_scheduler.step(val_loss)

        # Comprobar si este modelo es el mejor hasta ahora
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            if best_accuracy > global_best_accuracy:
                global_best_accuracy = best_accuracy
                # Guardar el nuevo valor de global_best_accuracy
                with open(best_accuracy_file, "w") as file:
                    json.dump(global_best_accuracy, file)
                torch.save(model.state_dict(), best_model_path)  # Sobreescribir el mejor modelo
                plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies, plot_file_name_base)  # Sobreescribir el gráfico
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve == early_stopping_patience:
            print("Early stopping")
            break

    print(f"Trial {trial.number} completado. Mejor precisión en validación: {best_accuracy}")

    # Limpiar la memoria después de cada trial
    del model, optimizer
    torch.cuda.empty_cache()
    gc.collect()

    # Devolver la pérdida en lugar de la precisión
    return best_accuracy


In [13]:
input_shape = features_train.shape[1]  # Número de variables explicativas

In [14]:
import optuna

storage = "sqlite:///example.db"  # Ruta de la base de datos SQLite
study_name = "accuracy_gpu_study"  # Nuevo nombre del estudio

# Crear un nuevo estudio
study = optuna.create_study(study_name=study_name, direction='maximize', storage=storage, load_if_exists=True)

/home/21953404Victor/SDC3/tutorial-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-12-01 16:41:35,444] Using an existing study with name 'accuracy_gpu_study' instead of creating a new one.


In [15]:
# Número de ensayos en el estudio
num_trials = len(study.trials)
print(f"El estudio contiene {num_trials} ensayos.")

El estudio contiene 1821 ensayos.


In [ ]:
trials_restantes=50000
study.optimize(objective, n_trials=trials_restantes)

Usando GPU para el Trial 1821
Entrenando Trial 1821...


[I 2023-12-01 16:42:16,950] Trial 1821 finished with value: 0.678407350689127 and parameters: {'n_layers': 38, 'n_units_l0': 16, 'activation_l0': 'LeakyReLU', 'dropout_l0': 0.032586957650862995, 'batch_norm_l0': False, 'l2_reg_l0': 4.322653215051211e-05, 'leakyrelu_alpha_l0': 0.13228233704565365, 'n_units_l1': 512, 'activation_l1': 'Tanh', 'dropout_l1': 0.2863276078865144, 'batch_norm_l1': False, 'l2_reg_l1': 3.1049299362087644e-05, 'n_units_l2': 1024, 'activation_l2': 'ReLU', 'dropout_l2': 0.2562823716463086, 'batch_norm_l2': True, 'l2_reg_l2': 0.014325419256705075, 'n_units_l3': 2048, 'activation_l3': 'LeakyReLU', 'dropout_l3': 0.44705868107466884, 'batch_norm_l3': False, 'l2_reg_l3': 0.04627729178200595, 'leakyrelu_alpha_l3': 0.2399670001095175, 'n_units_l4': 256, 'activation_l4': 'SELU', 'dropout_l4': 0.4290267911674831, 'batch_norm_l4': True, 'l2_reg_l4': 6.607234706628217e-05, 'n_units_l5': 2048, 'activation_l5': 'ReLU', 'dropout_l5': 0.027974032921513493, 'batch_norm_l5': True, 

Early stopping
Trial 1821 completado. Mejor precisión en validación: 0.678407350689127
Usando GPU para el Trial 1822
Entrenando Trial 1822...


[I 2023-12-01 16:42:34,575] Trial 1822 finished with value: 0.8741194486983155 and parameters: {'n_layers': 3, 'n_units_l0': 2048, 'activation_l0': 'LeakyReLU', 'dropout_l0': 0.06827115448687933, 'batch_norm_l0': False, 'l2_reg_l0': 0.001856826288716995, 'leakyrelu_alpha_l0': 0.10370000679135485, 'n_units_l1': 1024, 'activation_l1': 'ReLU', 'dropout_l1': 0.41102313164403365, 'batch_norm_l1': False, 'l2_reg_l1': 7.522848671317091e-05, 'n_units_l2': 1024, 'activation_l2': 'ReLU', 'dropout_l2': 0.15243215471841776, 'batch_norm_l2': True, 'l2_reg_l2': 0.0005380574788168477, 'optimizer': 'RMSprop', 'lr': 0.0023910720341213873, 'rmsprop_alpha': 0.906135199781518, 'epochs': 311, 'batch_size': 512, 'early_stopping_patience': 17, 'lr_reduce_factor': 0.44841867333731783, 'lr_reduce_patience': 37, 'min_lr': 1.493713537949615e-06}. Best is trial 506 with value: 0.8921898928024502.


Early stopping
Trial 1822 completado. Mejor precisión en validación: 0.8741194486983155
Usando GPU para el Trial 1823
Entrenando Trial 1823...


...

Early stopping
Trial 2041 completado. Mejor precisión en validación: 0.8551301684532925
Usando GPU para el Trial 2042
Entrenando Trial 2042...


[I 2023-12-01 20:47:34,148] Trial 2042 finished with value: 0.8318529862174578 and parameters: {'n_layers': 9, 'n_units_l0': 1024, 'activation_l0': 'LeakyReLU', 'dropout_l0': 0.0671467005613436, 'batch_norm_l0': False, 'l2_reg_l0': 0.01896005828489286, 'leakyrelu_alpha_l0': 0.24670743015837748, 'n_units_l1': 512, 'activation_l1': 'Tanh', 'dropout_l1': 0.13869665115778831, 'batch_norm_l1': False, 'l2_reg_l1': 0.023970542209017058, 'n_units_l2': 16, 'activation_l2': 'ReLU', 'dropout_l2': 0.3247956024335986, 'batch_norm_l2': True, 'l2_reg_l2': 1.2715903981344656e-05, 'n_units_l3': 2048, 'activation_l3': 'LeakyReLU', 'dropout_l3': 0.28975985913921354, 'batch_norm_l3': True, 'l2_reg_l3': 0.00034766764567648284, 'leakyrelu_alpha_l3': 0.16983290811910146, 'n_units_l4': 16, 'activation_l4': 'ELU', 'dropout_l4': 0.3528180296611436, 'batch_norm_l4': True, 'l2_reg_l4': 6.914467113130626e-05, 'n_units_l5': 16, 'activation_l5': 'LeakyReLU', 'dropout_l5': 0.3889271351937009, 'batch_norm_l5': False, 

Early stopping
Trial 2042 completado. Mejor precisión en validación: 0.8318529862174578
Usando GPU para el Trial 2043
Entrenando Trial 2043...


[I 2023-12-01 20:48:11,110] Trial 2043 finished with value: 0.863093415007657 and parameters: {'n_layers': 5, 'n_units_l0': 256, 'activation_l0': 'LeakyReLU', 'dropout_l0': 0.3252789004288207, 'batch_norm_l0': False, 'l2_reg_l0': 0.0004068952645918823, 'leakyrelu_alpha_l0': 0.279767277585379, 'n_units_l1': 512, 'activation_l1': 'Tanh', 'dropout_l1': 0.48829361514311387, 'batch_norm_l1': False, 'l2_reg_l1': 0.008366184194538748, 'n_units_l2': 1024, 'activation_l2': 'Tanh', 'dropout_l2': 0.3030802236678536, 'batch_norm_l2': True, 'l2_reg_l2': 9.872041404217726e-05, 'n_units_l3': 1024, 'activation_l3': 'ReLU', 'dropout_l3': 0.33378191953698283, 'batch_norm_l3': True, 'l2_reg_l3': 0.09942241130212895, 'n_units_l4': 64, 'activation_l4': 'ReLU', 'dropout_l4': 0.32391904085815826, 'batch_norm_l4': True, 'l2_reg_l4': 0.00019353655898471232, 'optimizer': 'RMSprop', 'lr': 0.002041719997677019, 'rmsprop_alpha': 0.9782504766198804, 'epochs': 1785, 'batch_size': 128, 'early_stopping_patience': 20, 

Early stopping
Trial 2043 completado. Mejor precisión en validación: 0.863093415007657
Usando GPU para el Trial 2044


In [ ]:
print('Número de pruebas finalizadas:', len(study.trials))
print('Mejor prueba:', study.best_trial.params)

## Test

In [ ]:
import torch.nn as nn

class PyTorchModel(nn.Module):
    def __init__(self, params, input_shape):
        super(PyTorchModel, self).__init__()
        self.layers = nn.ModuleList()
        self.l2_regs = []
        current_shape = input_shape

        # Definir la arquitectura de la red usando los hiperparámetros fijos
        n_layers = params['n_layers']
        for i in range(n_layers):
            num_units = params[f'n_units_l{i}']
            activation = params[f'activation_l{i}']
            dropout_rate = params[f'dropout_l{i}']
            use_batch_norm = params[f'batch_norm_l{i}']
            l2_reg = params[f'l2_reg_l{i}']

            # Crear y añadir la capa lineal
            layer = nn.Linear(current_shape, num_units)
            self.layers.append(layer)
            self.l2_regs.append(l2_reg)  # Añadir la tasa de regularización L2

            current_shape = num_units

            # Añadir Batch Normalization si se indica
            if use_batch_norm:
                self.layers.append(nn.BatchNorm1d(num_units))
            
            # Añadir Dropout
            self.layers.append(nn.Dropout(dropout_rate))

            # Añadir la activación
            if activation == 'LeakyReLU':
                negative_slope = params.get(f'leakyrelu_alpha_l{i}', 0.01)  # Valor predeterminado si no se proporciona
                self.layers.append(nn.LeakyReLU(negative_slope=negative_slope))
            else:
                self.layers.append(getattr(nn, activation)())

        # Capa de salida
        self.output = nn.Linear(current_shape, 1)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return torch.sigmoid(self.output(x))

    def get_l2_loss(self):
        l2_loss = 0.0
        for layer, l2_reg in zip(self.layers, self.l2_regs):
            if isinstance(layer, nn.Linear):
                l2_loss += l2_reg * layer.weight.norm(2)
        return l2_loss


In [ ]:
import torch

# Cargar el estado del modelo
state_dict = torch.load("best_model.pt")

# Imprimir la estructura del modelo
print(state_dict.keys())


In [ ]:
model = PyTorchModel(study.best_trial.params, input_shape)
model.load_state_dict(torch.load("best_model.pt"))
model.eval()  # Poner el modelo en modo de evaluación

In [ ]:
def evaluate_model(model, criterion, test_loader, device):
    model.eval()  # Poner el modelo en modo de evaluación
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # Desactivar el cálculo de gradientes
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            test_loss += criterion(outputs, labels).item()  # Sumar la pérdida del lote
            predicted = outputs.round()  # Redondear las salidas para obtener predicciones binarias
            correct += (predicted == labels).sum().item()  # Sumar predicciones correctas
            total += labels.size(0)

    test_loss /= len(test_loader)
    accuracy = correct / total
    return test_loss, accuracy

# Suponiendo que tienes un DataLoader para el conjunto de prueba llamado 'test_loader'
test_loss, test_accuracy = evaluate_model(model, criterion, test_loader, device)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

# Asegurarse de que el modelo esté en modo de evaluación y moverlo al dispositivo adecuado
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Realizar las predicciones
with torch.no_grad():
    inputs = features_test.to(device)
    outputs = model(inputs)
    predictions = outputs.round().cpu().numpy()  # Redondear las predicciones y moverlas al CPU

# Calcular y visualizar la matriz de confusión
conf_matrix = confusion_matrix(labels_test.numpy(), predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues')
plt.title('Matriz de Confusión')
plt.xlabel('Predicciones')
plt.ylabel('Valores Reales')
plt.show()

# Calcular y mostrar otras métricas
accuracy = accuracy_score(labels_test.numpy(), predictions)
recall = recall_score(labels_test.numpy(), predictions)
precision = precision_score(labels_test.numpy(), predictions)
f1 = f1_score(labels_test.numpy(), predictions)
print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")